In [8]:
import azureml.core
from preprocessing import get_workspace, get_hyperd_data, run_hyperd, \
    test_local_hyperd_model, register_and_deploy_hyperd_model, \
    test_deployed_hyperd_model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

IndentationError: unexpected indent (2936567938.py, line 3)

In [2]:
ws = get_workspace()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-199535
aml-quickstarts-199535
northcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


In [3]:
_, test_ds = get_hyperd_data()

Loading datasets from workspace ...


In [ ]:
model = run_hyperdrive()

In [5]:
test_local_hyperd_model(test_ds)

0.8318285117621768


0.8318285117621768

In [6]:
register_and_deploy_hyperd_model()

Registering model adult-hyperd-model
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-06-27 11:42:01+00:00 Creating Container Registry if not exists.
2022-06-27 11:42:01+00:00 Registering the environment.
2022-06-27 11:42:03+00:00 Use the existing image.
2022-06-27 11:42:03+00:00 Generating deployment configuration.
2022-06-27 11:42:04+00:00 Submitting deployment to compute.
2022-06-27 11:42:07+00:00 Checking the status of deployment adult-hyperd-service..
2022-06-27 11:44:12+00:00 Checking the status of inference endpoint adult-hyperd-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
/bin/bash: /azureml-envs/azureml_22f2115a88eed828c37333fb129fd861/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_22f2115a88eed828c37333fb129fd861/lib/libtinfo.so.6: no ve

In [7]:
for row in range(0, 10):
    test_deployed_hyperd_model(test_ds, row)

label: 0 prediction: 0
label: 0 prediction: 0
label: 1 prediction: 0
label: 1 prediction: 1
label: 0 prediction: 0
label: 0 prediction: 0
label: 0 prediction: 0
label: 1 prediction: 1
label: 0 prediction: 0
label: 0 prediction: 0
